In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
os.environ['OPENAI_API_KEY'] = "sk-OPENAI_API_KEY"


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Please respond to the user's request only based on the given context."),
    ("user", "Question: {question}\nContext: {context}")
])
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

question = "Can you summarize this morning's meetings?"
context = "During this morning's meeting, we solved all world conflict."
chain.invoke({"question": question, "context": context})

'The morning meeting successfully resolved all world conflicts.'

In [5]:
os.environ['LANGCHAIN_TRACING_V2'] = "True"
os.environ['LANGCHAIN_API_KEY'] = "LANGCHAIN_API_KEY"


In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Please respond to the user's request only based on the given context."),
    ("user", "Question: {question} \n Context: {context}")
])
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

question = "Can you summarize this morning's meetings?"
context = "During this morning's meeting, we solved all world conflict."
chain.invoke({"question": question, "context": context})

"During this morning's meeting, all world conflicts were resolved."

- We can wrap the OpenAI client:

In [7]:
from openai import OpenAI
from langsmith import wrappers

client = wrappers.wrap_openai(OpenAI())

In [8]:
client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": "Say this is a test"}],
)

ChatCompletion(id='chatcmpl-8uctwMwRixXAsq6H3MpY2PWWCzltz', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is a test.', role='assistant', function_call=None, tool_calls=None))], created=1708506596, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=5, prompt_tokens=12, total_tokens=17))

- traceable

In [9]:
from langsmith import traceable

@traceable
def my_function(text: str):
    return client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": f"Say {text}"}],
    )

my_function("hello world")

session_name: default my_function


ChatCompletion(id='chatcmpl-8ucuoQwEWo2l5Q1ctCqHqStqJ5sgK', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello World!', role='assistant', function_call=None, tool_calls=None))], created=1708506650, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=3, prompt_tokens=10, total_tokens=13))

- Instructor

In [10]:
from openai import OpenAI
from langsmith import wrappers

client = wrappers.wrap_openai(OpenAI())

In [12]:
pip install -U langsmith

Note: you may need to restart the kernel to use updated packages.


In [14]:
import instructor

client = instructor.patch(OpenAI())

In [15]:
from pydantic import BaseModel


class UserDetail(BaseModel):
    name: str
    age: int


user = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=UserDetail,
    messages=[
        {"role": "user", "content": "Extract Jason is 25 years old"},
    ]
)

In [16]:
# You can customize the run name with the `name` keyword argument
@traceable(name="Extract User Details")
def my_function(text: str) -> UserDetail:
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=UserDetail,
        messages=[
            {"role": "user", "content": f"Extract {text}"},
        ]
    )


my_function("Jason is 25 years old")

UserDetail(name='Jason', age=25)